# Agents
By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. 
Agents are systems that use an LLM as a reasoning enginer to determine which actions to take and what the inputs to those actions should be. 
The results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.

In [1]:
# Import relevant functionality
import os
import openai
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
openai.api_base = os.getenv('OPENAI_API_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")



In [2]:
# Create the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name="gpt4o",  # replace with your deployment name
    temperature=0.5,
    max_tokens=100,
    openai_api_version="2023-03-15-preview"
)

# Create the search tool
search = TavilySearchResults(max_results=2)
tools = [search]

# Create the agent with memory
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Configuration for conversation thread
config = {"configurable": {"thread_id": "abc123"}}



In [3]:
# testing the agent with a few messages

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="load the titanic.csv into a pandas dataframe")]}, config
):
    print(chunk)
    print("----")
    
    
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="how many rows are there in the titanic.csv?")]}, config
):
    print(chunk)
    print("----")
    
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="tell me more about the passengers")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have the capability to directly load files or interact with local file systems. However, I can certainly help you with the code to load the `titanic.csv` file into a pandas DataFrame. Here is an example of how you can do it:\n\n```python\nimport pandas as pd\n\n# Load the Titanic dataset\ndf = pd.read_csv('path_to_your_file/titanic.csv')\n\n# Display the first few rows of the dataframe\nprint(df.head())\n```\n\n", response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 88, 'total_tokens': 188}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5f4bad809a', 'finish_reason': 'length', 'logprobs': None, 'content_filter_results': {}}, id='run-c4b51a7c-2c9a-40fe-ad18-3bd0c2cf94a9-0', usage_metadata={'input_tokens': 88, 'output_tokens': 100, 'total_tokens': 188})]}}
----
{'agent': {'messages': [AIMessage(content="To find out the number of rows in the `titanic.csv` file, you can use the following Python code:\

[]


TypeError: Threads.create() got an unexpected keyword argument 'assistant_id'